In [9]:
import base64
import mimetypes
import os
from google import genai
from google.genai import types
import dotenv

dotenv.load_dotenv()


project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
location = "global"
print(f"Project ID: {project_id}, Location: {location}")    


def save_binary_file(file_name, data):
    """Saves binary data to a file."""
    f = open(file_name, "wb")
    f.write(data)
    f.close()
    print(f"File saved to: {file_name}")

def load_image_bytes(file_path):
    """Loads a local image file and returns its bytes and MIME type."""
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        raise ValueError(f"Could not determine MIME type for {file_path}")
    
    with open(file_path, 'rb') as f:
        image_bytes = f.read()
    
    return image_bytes, mime_type

In [22]:
client = genai.Client(
    # api_key=os.environ.get("GEMINI_API_KEY"),
    vertexai=True,
    project=project_id,
    location=location
)

model = "gemini-2.5-flash-image-preview"

# --- Change 1: Load multiple images from local files ---
# Replace these paths with the actual paths to your image files.
try:
    image1_path = "./images/s-l1200.jpg"  # e.g., "path/to/a/product.jpg"
    image2_path = "./images/20250921_183221_254e5c80.jpg" # e.g., "path/to/a/background_scene.png"
    
    image1_bytes, image1_mime_type = load_image_bytes(image1_path)
    image2_bytes, image2_mime_type = load_image_bytes(image2_path)
except FileNotFoundError as e:
    print(f"Error: {e}. Please update the file paths to your images.")

In [20]:

# --- Change 2: Create a multimodal contents list with text and images ---
# The order of parts matters. The text prompt can be anywhere in the list.
contents = [
    types.Content(
        role="user",
        parts=[
            # Provide the first image as a part
            types.Part.from_bytes(data=image1_bytes, mime_type=image1_mime_type),
            # Provide the second image as a part
            types.Part.from_bytes(data=image2_bytes, mime_type=image2_mime_type),
            # Provide the text prompt to describe the desired composition
            types.Part.from_text(text="""Place the poster from Image 1 onto the right wall of Image 2 above the pillows, sized like a 24×36 print. Match wall perspective and warm LED lighting; add a soft top-edge glow and faint contact shadows on the right/bottom edges. Remove borders, keep text sharp, preserve all decor. Output a high-resolution, seamless composite"""),
        ],
    ),
]

generate_content_config = types.GenerateContentConfig(
    response_modalities=[
        "IMAGE",
        "TEXT",
    ],
)

In [23]:
file_index = 0
print("Generating composite image...")
for chunk in client.models.generate_content_stream(
    model=model,
    contents=contents,
    config=generate_content_config,
):
    if (
        chunk.candidates is None
        or chunk.candidates[0].content is None
        or chunk.candidates[0].content.parts is None
    ):
        continue
    if chunk.candidates[0].content.parts[0].inline_data and chunk.candidates[0].content.parts[0].inline_data.data:
        file_name = f"composite_image_{file_index}"
        file_index += 1
        inline_data = chunk.candidates[0].content.parts[0].inline_data
        data_buffer = inline_data.data
        file_extension = mimetypes.guess_extension(inline_data.mime_type)
        save_binary_file(f"{file_name}{file_extension}", data_buffer)
    else:
        print(chunk.text)

Generating composite image...


ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'Publisher Model `projects/gke-hack-471804/locations/global/publishers/google/models/gemini-1.5-pro-preview-0514` not found.', 'status': 'NOT_FOUND'}}